## **1. Import pandas as pd**


Csv file loaded into pandas dataframe


First 10 rows of the dataset shown

In [2]:
import pandas as pd

house_sales = pd.read_csv("kc_house_data.csv")
print(house_sales[:10])


           id             date      price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000   221900.0         3       1.00         1180   
1  6414100192  20141209T000000   538000.0         3       2.25         2570   
2  5631500400  20150225T000000   180000.0         2       1.00          770   
3  2487200875  20141209T000000   604000.0         4       3.00         1960   
4  1954400510  20150218T000000   510000.0         3       2.00         1680   
5  7237550310  20140512T000000  1225000.0         4       4.50         5420   
6  1321400060  20140627T000000   257500.0         3       2.25         1715   
7  2008000270  20150115T000000   291850.0         3       1.50         1060   
8  2414600126  20150415T000000   229500.0         3       1.00         1780   
9  3793500160  20150312T000000   323000.0         3       2.50         1890   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650     1.0           0     0  ...      7 

## **2. Displaying basic information about house_sales dataset**

In [3]:
house_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

## **3. Data Cleaning**

#### 3.1. Identifying and handling missing values

From the previous code snippet where the description of the dataframe is shown, we can see that there are a total of 21613 rows of data. Out of the 21613, there are no null values in the columns.

Some columns have the value zero, which explains that the house in question doesn't have that specific feature. So, if the year of renovation is zero, it means the house sold was never renovated.

So, there not being any missing values, I will keep the dataframe unchanged.

#### 3.2. Checking and removing duplicate records

In [31]:
duplicate = house_sales['view'].value_counts()
print(duplicate)

view
0    19489
2      963
3      510
1      332
4      319
Name: count, dtype: int64
